**Command to kill the tunnel of the ngrok**
**you need to run it after 3 times running the Main code**


```
# This is formatted as code
```





In [ ]:
!killall ngrok

ngrok: no process found


**Main Code**

In [ ]:
!pip install flask
!pip install pyngrok
!pip install requests
!pip install pandas
!pip install plotly

In [ ]:
app_code = """
from flask import Flask, render_template, request, redirect, url_for, jsonify
from werkzeug.utils import secure_filename
import pandas as pd
import json
import requests
from datetime import datetime
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import os
from collections import Counter
import re

app = Flask(__name__)

firebase_url = "https://cloud-a87bf-default-rtdb.firebaseio.com"  # Replace with your Firebase Realtime Database URL
collection_name = "data"  # Centralized collection name

def fetch_from_firebase():
    url = f"{firebase_url}/{collection_name}.json"
    response = requests.get(url)
    return response.json() if response.ok else {}

def upload_to_firebase(data):
    url = f"{firebase_url}/{collection_name}.json"
    response = requests.post(url, json=data)
    return response.json()

def delete_collection():
    url = f"{firebase_url}/{collection_name}.json"
    requests.delete(url)

def get_unique_values(data, key):
    return sorted(set(entry.get(key, '') for entry in data.values() if entry.get(key, '')))

def analyze_days_off(user, data):
    user_actions = [entry for entry in data.values() if entry["User"] == user]
    if not user_actions:
        return None

    df = pd.DataFrame(user_actions)
    df['Time'] = pd.to_datetime(df['Time'])
    df['Date'] = df['Time'].dt.date

    daily_actions = df.groupby('Date').size()
    days_off = daily_actions[daily_actions < 50]

    return days_off

def analyze_user_actions_by_document(user, data):
    user_actions = [entry for entry in data.values() if entry["User"] == user]
    document_actions = {}

    for action in user_actions:
        document = action["Document"]
        if document not in document_actions:
            document_actions[document] = {'total': 0, 'delete': 0}
        document_actions[document]['total'] += 1
        if 'delete' in action['Description'].lower():
            document_actions[document]['delete'] += 1

    for doc in document_actions:
        total = document_actions[doc]['total']
        delete = document_actions[doc]['delete']
        document_actions[doc]['percentage'] = (delete / total) * 100 if total > 0 else 0

    return document_actions

def analyze_user_progress(user, data):
    user_actions = [entry for entry in data.values() if entry["User"] == user]
    if not user_actions:
        return None, None

    relevant_actions = [entry for entry in user_actions if 'delete' not in entry['Description'].lower()]
    df = pd.DataFrame(relevant_actions)
    df['Time'] = pd.to_datetime(df['Time'])
    df.sort_values(by='Time', inplace=True)

    df['Month'] = df['Time'].dt.to_period('M')
    monthly_actions = df.groupby('Month').size()

    return df, monthly_actions

# Home route
@app.route('/')
def home():
    team_members = [
        {"name": "Mohamad Abu Ahmad", "id": 314934613},
        {"name": "Shady Mansour", "id": 2},
        {"name": "Majd Salameh", "id": 3},
        {"name": "Jacob Sadran", "id": 4},
        {"name": "Eyas Rizik", "id": 5}
    ]
    return render_template('index.html', team_members=team_members)

# Word Frequency and Search route
def word_frequency(data):
    word_count = Counter()
    stop_words = set([
        "a", "about", "above", "after", "again", "against", "all", "am", "an", "and", "any", "are", "aren't", "as", "at",
        "be", "because", "been", "before", "being", "below", "between", "both", "but", "by", "can't", "cannot", "could",
        "couldn't", "did", "didn't", "do", "does", "doesn't", "doing", "don't", "down", "during", "each", "few", "for",
        "from", "further", "had", "hadn't", "has", "hasn't", "have", "haven't", "having", "he", "he'd", "he'll", "he's",
        "her", "here", "here's", "hers", "herself", "him", "himself", "his", "how", "how's", "i", "i'd", "i'll", "i'm",
        "i've", "if", "in", "into", "is", "isn't", "it", "it's", "its", "itself", "let's", "me", "more", "most", "mustn't",
        "my", "myself", "no", "nor", "not", "of", "off", "on", "once", "only", "or", "other", "ought", "our", "ours",
        "ourselves", "out", "over", "own", "same", "shan't", "she", "she'd", "she'll", "she's", "should", "shouldn't",
        "so", "some", "such", "than", "that", "that's", "the", "their", "theirs", "them", "themselves", "then", "there",
        "there's", "these", "they", "they'd", "they'll", "they're", "they've", "this", "those", "through", "to", "too",
        "under", "until", "up", "very", "was", "wasn't", "we", "we'd", "we'll", "we're", "we've", "were", "weren't", "what",
        "what's", "when", "when's", "where", "where's", "which", "while", "who", "who's", "whom", "why", "why's", "with",
        "won't", "would", "wouldn't", "you", "you'd", "you'll", "you're", "you've", "your", "yours", "yourself", "yourselves"
    ])

    for entry in data.values():
        description = entry.get('Description', '').lower()
        words = re.findall(r'\w+', description)
        filtered_words = [word for word in words if word not in stop_words]
        word_count.update(filtered_words)

    return word_count.most_common()

@app.route('/word_frequency', methods=['GET', 'POST'])
def word_frequency_route():
    data = fetch_from_firebase()
    if data is None:
        data = {}

    word_freq = word_frequency(data)
    search_results = []
    word_count = 0
    search_query = ""

    if request.method == 'POST':
        search_query = request.form.get('search_query')
        for entry in data.values():
            if search_query.lower() in entry.get('Description', '').lower():
                search_results.append(entry)
                word_count += entry.get('Description', '').lower().count(search_query.lower())

    return render_template('word_frequency.html', word_freq=word_freq, search_results=search_results, word_count=word_count, search_query=search_query)

@app.route('/parameters', methods=['GET', 'POST'])
def parameters():
    data = fetch_from_firebase()
    if data is None:
        data = {}

    users = sorted(set([entry['User'] for entry in data.values()]))
    documents = sorted(set([entry['Document'] for entry in data.values()]))
    descriptions = sorted(set([entry['Description'] for entry in data.values()]))
    tabs = sorted(set([entry['Tab'] for entry in data.values()]))

    if request.method == 'POST':
        user = request.form.get('user')
        document = request.form.get('document')
        description = request.form.get('description')
        tab = request.form.get('tab')
        date = request.form.get('date')
        return redirect(url_for('results', user=user, document=document, description=description, tab=tab, date=date))

    return render_template('parameters.html', users=users, documents=documents, descriptions=descriptions, tabs=tabs)

@app.route('/results')
def results():
    user = request.args.get('user')
    document = request.args.get('document')
    description = request.args.get('description')
    tab = request.args.get('tab')
    date = request.args.get('date')
    data = fetch_from_firebase()
    if data is None:
        data = {}

    filtered_data = [
        entry for entry in data.values() if
        (not user or user in entry.get('User', '')) and
        (not document or document in entry.get('Document', '')) and
        (not description or description in entry.get('Description', '')) and
        (not tab or tab in entry.get('Tab', '')) and
        (not date or date in entry.get('Time', ''))
    ]

    document_actions = analyze_user_actions_by_document(user, data)
    df, monthly_actions = analyze_user_progress(user, data)
    days_off = analyze_days_off(user, data)

    graphs = []

    fig = make_subplots(
        rows=2, cols=2,
        subplot_titles=(
            "Total Actions per Document",
            "Delete Actions per Document",
            "Percentage of Delete Actions per Document",
            "Monthly User Progress"
        )
    )

    if document_actions:
        documents = list(document_actions.keys())
        total_actions = [document_actions[doc]['total'] for doc in documents]
        delete_actions = [document_actions[doc]['delete'] for doc in documents]
        delete_percentages = [document_actions[doc]['percentage'] for doc in documents]

        fig.add_trace(go.Bar(x=documents, y=total_actions, name='Total Actions'), row=1, col=1)
        fig.add_trace(go.Bar(x=documents, y=delete_actions, name='Delete Actions'), row=1, col=2)
        fig.add_trace(go.Bar(x=documents, y=delete_percentages, name='Delete Percentage'), row=2, col=1)

    if df is not None and monthly_actions is not None:
        fig.add_trace(go.Scatter(x=monthly_actions.index.astype(str), y=monthly_actions.values, mode='lines+markers', name='Line Chart'), row=2, col=2)
        fig.add_trace(go.Bar(x=monthly_actions.index.astype(str), y=monthly_actions.values, name='Bar Chart'), row=2, col=2)

    if days_off is not None:
        fig_days_off = go.Figure()
        fig_days_off.add_trace(go.Bar(x=days_off.index, y=days_off.values, name='Days Off'))
        fig_days_off.update_layout(
            title_text="Number of Days Off (Less than 50 Actions)",
            title_font_size=20,
            title_font_family="Arial",
            showlegend=True
        )
        graphs.append(fig_days_off)

    fig.update_layout(
        height=800, width=1200,
        title_text="User Analysis",
        title_font_size=20,
        title_font_family="Arial",
        showlegend=False
    )

    fig.update_annotations(font_size=16, font_family="Arial", font=dict(color="black", family="Arial", size=14))

    graphs.append(fig)

    return render_template('results.html', data=filtered_data, graphs=graphs)

# Endpoint to serve data for results page
@app.route('/data')
def get_data():
    data = fetch_from_firebase()
    if data is None:
        data = {}
    return jsonify(data)

# Interesting Statistics Screen
@app.route('/statistics')
def statistics():
    data = fetch_from_firebase()
    if data is None:
        data = {}
    total_entries = len(data)
    unique_users = len(set([item['User'] for item in data.values()]))
    unique_documents = len(set([item['Document'] for item in data.values()]))
    statistics_data = {
        'total_entries': total_entries,
        'unique_users': unique_users,
        'unique_documents': unique_documents,
    }
    return render_template('statistics.html', statistics=statistics_data)

# Admin Screen for Uploading JSON
@app.route('/upload', methods=['GET', 'POST'])
def upload():
    if request.method == 'POST':
        file = request.files['file']
        filename = secure_filename(file.filename)
        filepath = os.path.join('/content', filename)
        file.save(filepath)
        with open(filepath) as json_file:
            data = json.load(json_file)
            delete_collection()  # Clear existing collection
            for item in data:
                upload_to_firebase(item)
        return redirect(url_for('upload'))
    return render_template('upload.html')

# Contact Us Screen
@app.route('/contact', methods=['GET', 'POST'])
def contact():
    if request.method == 'POST':
        name = request.form.get('name')
        email = request.form.get('email')
        message = request.form.get('message')
        # Process the form data as needed
        print(f"Received message from {name} ({email}): {message}")
        return redirect(url_for('contact'))
    return render_template('contact.html')

# Search Screen
@app.route('/search', methods=['GET', 'POST'])
def search():
    data = fetch_from_firebase()
    if data is None:
        data = {}

    search_results = []
    search_query = ""

    if request.method == 'POST':
        search_query = request.form.get('search_query')
        for entry in data.values():
            if search_query.lower() in entry.get('Description', '').lower():
                search_results.append(entry)

    return render_template('search.html', search_results=search_results, search_query=search_query)

if __name__ == '__main__':
    app.run(debug=True, host='0.0.0.0')

"""

# Write the app_code to a file
with open('app.py', 'w') as f:
    f.write(app_code)

# Create templates directory and HTML files
!mkdir -p templates

# index.html
index_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Home</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="sidebar">
        <img src="https://upload.wikimedia.org/wikipedia/commons/4/45/Onshape_logo_full.png" alt="Logo" class="logo">
        <a href="/">Home</a>
        <a href="/word_frequency">Word Frequency & Search</a>
        <a href="/parameters">Parameter Selection</a>
        <a href="/results">Parameter Results</a>
        <a href="/statistics">Statistics</a>
        <a href="/upload">Upload JSON</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="content">
        <h1>Welcome to the Data Analysis Hub</h1>
        <p>Unlock the power of your data with our intuitive and comprehensive analysis tools. Whether you're tracking user activities or monitoring document interactions, our platform provides you with the insights you need to make informed decisions.</p>
        <p>At the heart of our application lies a commitment to excellence and innovation. Our team of dedicated developers has worked tirelessly to create a seamless experience that combines advanced analytics with user-friendly design.</p>
        <p><strong>Meet the Team:</strong></p>
        <ul>
            <li><strong>Mohamad Abu Ahmad</strong> - The visionary behind the data integration and user interface.</li>
            <li><strong>Shady Mansour</strong> - Our coding maestro who brought the backend to life.</li>
            <li><strong>Majd Salameh</strong> - The mastermind of data visualization and interactive graphs.</li>
            <li><strong>Jacob Sadran</strong> - The architect of seamless user experience and design aesthetics.</li>
            <li><strong>Eyas Rizik</strong> - The quality assurance guru ensuring everything runs smoothly.</li>
        </ul>
        <p>Join us on a journey of data discovery and let our platform transform the way you see your information. Dive in, explore, and make the most of your data with our state-of-the-art tools and features.</p>
    </div>
</body>
</html>
"""

# parameters.html
parameters_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Parameter Selection</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="sidebar">
        <img src="https://upload.wikimedia.org/wikipedia/commons/4/45/Onshape_logo_full.png" alt="Logo" class="logo">
        <a href="/">Home</a>
        <a href="/word_frequency">Word Frequency & Search</a>
        <a href="/parameters">Parameter Selection</a>
        <a href="/results">Parameter Results</a>
        <a href="/statistics">Statistics</a>
        <a href="/upload">Upload JSON</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="content">
        <h1>Select Parameters</h1>
        <form method="POST" action="/parameters">
            <label for="user">User:</label>
            <select id="user" name="user">
                <option value="">Select User</option>
                {% for user in users %}
                    <option value="{{ user }}">{{ user }}</option>
                {% endfor %}
            </select>

            <label for="document">Document:</label>
            <select id="document" name="document">
                <option value="">Select Document</option>
                {% for document in documents %}
                    <option value="{{ document }}">{{ document }}</option>
                {% endfor %}
            </select>

            <label for="description">Description:</label>
            <select id="description" name="description">
                <option value="">Select Description</option>
                {% for description in descriptions %}
                    <option value="{{ description }}">{{ description }}</option>
                {% endfor %}
            </select>

            <label for="tab">Tab:</label>
            <select id="tab" name="tab">
                <option value="">Select Tab</option>
                {% for tab in tabs %}
                    <option value="{{ tab }}">{{ tab }}</option>
                {% endfor %}
            </select>

            <label for="date">Date:</label>
            <input type="date" id="date" name="date">

            <button type="submit">Submit</button>
        </form>
    </div>
</body>
</html>

"""

# results.html
results_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Parameter Results</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="sidebar">
        <img src="https://upload.wikimedia.org/wikipedia/commons/4/45/Onshape_logo_full.png" alt="Logo" class="logo">
        <a href="/">Home</a>
        <a href="/word_frequency">Word Frequency & Search</a>
        <a href="/parameters">Parameter Selection</a>
        <a href="/results">Parameter Results</a>
        <a href="/statistics">Statistics</a>
        <a href="/upload">Upload JSON</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="content">
        <div id="graphs">
            {% for fig in graphs %}
                <div>{{ fig.to_html(full_html=False) | safe }}</div>
            {% endfor %}
        </div>
        <h1>Parameter Results</h1>
        <table>
            <thead>
                <tr>
                    <th>Time</th>
                    <th>Document</th>
                    <th>Tab</th>
                    <th>User</th>
                    <th>Description</th>
                </tr>
            </thead>
            <tbody id="results-table-body">
                {% for item in data %}
                <tr>
                    <td>{{ item.Time }}</td>
                    <td>{{ item.Document }}</td>
                    <td>{{ item.Tab }}</td>
                    <td>{{ item.User }}</td>
                    <td>{{ item.Description }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
    </div>
    <script src="{{ url_for('static', filename='script.js') }}"></script>
</body>
</html>
"""

# statistics.html
statistics_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Statistics</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="sidebar">
        <img src="https://upload.wikimedia.org/wikipedia/commons/4/45/Onshape_logo_full.png" alt="Logo" class="logo">
        <a href="/">Home</a>
        <a href="/word_frequency">Word Frequency & Search</a>
        <a href="/parameters">Parameter Selection</a>
        <a href="/results">Parameter Results</a>
        <a href="/statistics">Statistics</a>
        <a href="/upload">Upload JSON</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="content">
        <h1>Interesting Statistics</h1>
        <p>Total Entries: {{ statistics.total_entries }}</p>
        <p>Unique Users: {{ statistics.unique_users }}</p>
        <p>Unique Documents: {{ statistics.unique_documents }}</p>
    </div>
</body>
</html>
"""

# upload.html
upload_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Upload JSON</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="sidebar">
        <img src="https://upload.wikimedia.org/wikipedia/commons/4/45/Onshape_logo_full.png" alt="Logo" class="logo">
        <a href="/">Home</a>
        <a href="/word_frequency">Word Frequency & Search</a>
        <a href="/parameters">Parameter Selection</a>
        <a href="/results">Parameter Results</a>
        <a href="/statistics">Statistics</a>
        <a href="/upload">Upload JSON</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="content">
        <h1>Upload JSON File</h1>
        <form method="POST" action="/upload" enctype="multipart/form-data">
            <label for="file">Choose JSON file:</label>
            <input type="file" id="file" name="file" accept=".json">
            <button type="submit">Upload</button>
        </form>
    </div>
</body>
</html>
"""

# search.html
search_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Search</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="sidebar">
        <img src="https://upload.wikimedia.org/wikipedia/commons/4/45/Onshape_logo_full.png" alt="Logo" class="logo">
        <a href="/">Home</a>
        <a href="/word_frequency">Word Frequency & Search</a>
        <a href="/parameters">Parameter Selection</a>
        <a href="/results">Parameter Results</a>
        <a href="/statistics">Statistics</a>
        <a href="/upload">Upload JSON</a>
        <a href="/search">Search</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="content">
        <h1>Search OnShape Index</h1>
        <form method="POST" action="/search">
            <label for="search_query">Enter word to search:</label>
            <input type="text" id="search_query" name="search_query" required>
            <button type="submit">Search</button>
        </form>

        {% if search_query %}
            <h2>Search Results for "{{ search_query }}"</h2>
            <table>
                <thead>
                    <tr>
                        <th>Time</th>
                        <th>Document</th>
                        <th>Tab</th>
                        <th>User</th>
                        <th>Description</th>
                    </tr>
                </thead>
                <tbody>
                    {% for item in search_results %}
                        <tr>
                            <td>{{ item.Time }}</td>
                            <td>{{ item.Document }}</td>
                            <td>{{ item.Tab }}</td>
                            <td>{{ item.User }}</td>
                            <td>{{ item.Description }}</td>
                        </tr>
                    {% endfor %}
                </tbody>
            </table>
        {% endif %}
    </div>
</body>
</html>
"""
word_frequency_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Word Frequency & Search</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="sidebar">
        <img src="https://upload.wikimedia.org/wikipedia/commons/4/45/Onshape_logo_full.png" alt="Logo" class="logo">
        <a href="/">Home</a>
        <a href="/word_frequency">Word Frequency & Search</a>
        <a href="/parameters">Parameter Selection</a>
        <a href="/results">Parameter Results</a>
        <a href="/statistics">Statistics</a>
        <a href="/upload">Upload JSON</a>
        <a href="/search">Search</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="content">
        <h1>Word Frequency Analysis</h1>
        <table>
            <thead>
                <tr>
                    <th>Term</th>
                    <th>Freq</th>
                </tr>
            </thead>
            <tbody>
                {% for word, count in word_freq %}
                <tr>
                    <td>{{ word }}</td>
                    <td>{{ count }}</td>
                </tr>
                {% endfor %}
            </tbody>
        </table>
        <h2>Search for a Word</h2>
        <form method="POST" action="/word_frequency">
            <label for="search_query">Enter word to search:</label>
            <input type="text" id="search_query" name="search_query" required>
            <button type="submit">Search</button>
        </form>

        {% if search_query %}
            <h2>Search Results for "{{ search_query }}"</h2>
            <p>Word Count: {{ word_count }}</p>
            <table>
                <thead>
                    <tr>
                        <th>Time</th>
                        <th>Document</th>
                        <th>Tab</th>
                        <th>User</th>
                        <th>Description</th>
                    </tr>
                </thead>
                <tbody>
                    {% for item in search_results %}
                        <tr>
                            <td>{{ item.Time }}</td>
                            <td>{{ item.Document }}</td>
                            <td>{{ item.Tab }}</td>
                            <td>{{ item.User }}</td>
                            <td>{{ item.Description }}</td>
                        </tr>
                    {% endfor %}
                </tbody>
            </table>
        {% endif %}
    </div>
</body>
</html>
"""

contactus_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Contact Us</title>
    <link rel="stylesheet" href="{{ url_for('static', filename='style.css') }}">
</head>
<body>
    <div class="sidebar">
        <img src="https://upload.wikimedia.org/wikipedia/commons/4/45/Onshape_logo_full.png" alt="Logo" class="logo">
        <a href="/">Home</a>
        <a href="/word_frequency">Word Frequency & Search</a>
        <a href="/parameters">Parameter Selection</a>
        <a href="/results">Parameter Results</a>
        <a href="/statistics">Statistics</a>
        <a href="/upload">Upload JSON</a>
        <a href="/contact">Contact Us</a>
    </div>
    <div class="content">
        <h1>Contact Us</h1>
        <form method="POST" action="/contact">
            <label for="name">Name:</label>
            <input type="text" id="name" name="name" required>
            <label for="email">Email:</label>
            <input type="email" id="email" name="email" required>
            <label for="message">Message:</label>
            <textarea id="message" name="message" rows="5" required></textarea>
            <button type="submit">Send</button>
        </form>
    </div>
</body>
</html>

"""

# Write HTML files to the templates directory

with open('templates/word_frequency.html', 'w') as f:
    f.write(word_frequency_html)

with open('templates/index.html', 'w') as f:
    f.write(index_html)

with open('templates/parameters.html', 'w') as f:
    f.write(parameters_html)

with open('templates/results.html', 'w') as f:
    f.write(results_html)

with open('templates/statistics.html', 'w') as f:
    f.write(statistics_html)

with open('templates/upload.html', 'w') as f:
    f.write(upload_html)

with open('templates/search.html', 'w') as f:
    f.write(search_html)

with open('templates/contact.html', 'w') as f:
    f.write(contactus_html)

# Create static directory and files
!mkdir -p static

# style.css
style_css = """
body {
    font-family: Arial, sans-serif;
    margin: 0;
    padding: 0;
    display: flex;
}

.sidebar {
    width: 200px;
    background-color: #333;
    color: #fff;
    padding: 15px;
    height: 100vh;
    position: fixed;
}

.sidebar a {
    color: #fff;
    text-decoration: none;
    display: block;
    margin: 10px 0;
}

.logo {
    width: 100%;
    margin-bottom: 20px;
}

.content {
    margin-left: 220px;
    padding: 20px;
}

form label {
    display: block;
    margin: 10px 0 5px;
}

form input, form textarea, form select {
    width: 100%;
    padding: 10px;
    margin-bottom: 20px;
    border: 1px solid #ccc;
    border-radius: 5px;
}

form button {
    padding: 10px 20px;
    background-color: #28a745;
    color: #fff;
    border: none;
    border-radius: 5px;
    cursor: pointer;
}

table {
    width: 100%;
    border-collapse: collapse;
    margin-top: 20px;
}

table, th, td {
    border: 1px solid #ccc;
}

th, td {
    padding: 10px;
    text-align: left;
}
"""

# script.js
script_js = """
// Fetch data and display on the results page
if (window.location.pathname === '/results') {
    fetch('/data')
        .then(response => response.json())
        .then(data => {
            const tableBody = document.getElementById('results-table-body');
            data.forEach(item => {
                const row = document.createElement('tr');
                row.innerHTML = `
                    <td>${item.Time}</td>
                    <td>${item.Document}</td>
                    <td>${item.Tab}</td>
                    <td>${item.User}</td>
                    <td>${item.Description}</td>
                `;
                tableBody.appendChild(row);
            });
        });
}
"""

with open('static/style.css', 'w') as f:
    f.write(style_css)

with open('static/script.js', 'w') as f:
    f.write(script_js)



# Set Ngrok auth token and start the server
from pyngrok import ngrok

# Use your new authtoken here
ngrok.set_auth_token("2jNcgPBXwAf1va24BQI4NIKiTQq_76R2eftdY2tWW18CmSH42")

# Start Flask app in the background
get_ipython().system_raw('python3 app.py &')

# Pause to ensure Flask app starts
import time
time.sleep(5)

# Start Ngrok tunnel
public_url = ngrok.connect(5000)
print('Public URL:', public_url)


Public URL: NgrokTunnel: "https://3f6c-34-83-92-115.ngrok-free.app" -> "http://localhost:5000"
